In [6]:
import os
import numpy as np
from PIL import Image
import random
import tensorflow as tf
from tensorflow.keras import layers, models

# Step 1: Data Collection and Preprocessing
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = Image.open(os.path.join(folder_path, filename))
        if img is not None:
            images.append(img)
    return images

def resize_and_normalize(images, target_size=(128, 128)):
    processed_images = []
    for img in images:
        img = img.resize(target_size, Image.ANTIALIAS)
        img_array = np.array(img)
        # Normalize pixel values to [-1, 1]
        normalized_img = (img_array.astype(np.float32) - 127.5) / 127.5
        processed_images.append(normalized_img)
    return np.array(processed_images)

def split_data(data, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1, random_seed=42):
    random.seed(random_seed)
    num_samples = len(data)
    train_size = int(train_ratio * num_samples)
    val_size = int(val_ratio * num_samples)

    random.shuffle(data)
    train_data = data[:train_size]
    remaining_data = data[train_size:]

    val_data = remaining_data[:val_size]
    test_data = remaining_data[val_size:]

    return train_data, val_data, test_data

raw_dataset_folder = 'cars'
category_images = load_images_from_folder(raw_dataset_folder)

# Resize and normalize the images to 128x128
target_size = (128, 128)
processed_images = resize_and_normalize(category_images, target_size=target_size)

# Split the data into training, validation, and testing sets
train_data, val_data, test_data = split_data(processed_images)

# Print the number of images in each set
print(f"Number of training images: {len(train_data)}")
print(f"Number of validation images: {len(val_data)}")
print(f"Number of testing images: {len(test_data)}")


ModuleNotFoundError: No module named 'PIL'

In [ ]:
from tensorflow.keras import layers, models

# Step 2: Define the Generator and Discriminator Networks

# Generator architecture
def make_generator_model(noise_dim=100, target_size=(128, 128, 3)):
    model = models.Sequential()
    model.add(layers.Dense(256, input_dim=noise_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization())

    model.add(layers.Dense(np.prod(target_size), activation='tanh'))
    model.add(layers.Reshape(target_size))

    return model

# Discriminator architecture
def make_discriminator_model(target_size=(128, 128, 3)):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=target_size))

    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))

    model.add(layers.Dense(1, activation='sigmoid'))

    return model

# Create instances of the generator and discriminator models
generator = make_generator_model()
discriminator = make_discriminator_model()

# Print the summary of generator and discriminator models
generator.summary()
discriminator.summary()


In [ ]:
import tensorflow as tf

# Step 3: Build the GAN Model

def make_gan_model(generator, discriminator):
    discriminator.trainable = False
    gan_input = tf.keras.Input(shape=(100,))
    x = generator(gan_input)
    gan_output = discriminator(x)

    gan = tf.keras.models.Model(gan_input, gan_output)
    return gan

# Create the GAN model
gan = make_gan_model(generator, discriminator)

# Compile the GAN (for generator training)
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))


In [ ]:
import matplotlib.pyplot as plt

# Step 4: Define Loss Functions and Implement Training Loop

# Define loss functions for generator and discriminator
def generator_loss(fake_output):
    return tf.keras.losses.binary_crossentropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.binary_crossentropy(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# Training parameters
EPOCHS = 100
BATCH_SIZE = 128
NOISE_DIM = 100

# Function to generate and save sample images during training
def generate_and_save_images(model, epoch, test_input, save_path='gan_samples'):
    predictions = model(test_input, training=False)
    predictions = (predictions + 1) * 0.5  # Rescale images to [0, 1]
    fig = plt.figure(figsize=(8, 8))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')
    plt.savefig(f'images/image_at_epoch_{epoch:04d}.png')
    plt.close()

# Training loop
def train_gan(generator, discriminator, gan, train_data, epochs, batch_size, noise_dim):
    num_batches = len(train_data) // batch_size
    for epoch in range(epochs):
        for batch_idx in range(num_batches):
            # Train discriminator
            noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
            generated_images = generator.predict(noise)

            real_images = train_data[batch_idx * batch_size: (batch_idx + 1) * batch_size]

            real_labels = np.ones((batch_size, 1))
            fake_labels = np.zeros((batch_size, 1))

            d_loss_real = discriminator.train_on_batch(real_images, real_labels)
            d_loss_fake = discriminator.train_on_batch(generated_images, fake_labels)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train generator
            noise = np.random.normal(0, 1, size=(batch_size, noise_dim))
            valid_labels = np.ones((batch_size, 1))
            g_loss = gan.train_on_batch(noise, valid_labels)


In [ ]:
# Step 5: Start GAN Training and Evaluate Progress

# Training parameters
EPOCHS = 100
BATCH_SIZE = 128
NOISE_DIM = 100

# Create a new instance of the generator and discriminator models
generator = make_generator_model()
discriminator = make_discriminator_model()

# Compile the discriminator (for binary classification)
discriminator.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Create the GAN model
gan = make_gan_model(generator, discriminator)

# Compile the GAN (for generator training)
gan.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

# Step 5: Start GAN Training
train_gan(generator, discriminator, gan, train_data, EPOCHS, BATCH_SIZE, NOISE_DIM)

In [ ]:
# Step 6: Evaluate and Fine-Tune (Optional)

# After the GAN training is complete, you can generate and save new images for evaluation:
num_evaluation_samples = 16
evaluation_noise = np.random.normal(0, 1, size=(num_evaluation_samples, NOISE_DIM))
generate_and_save_images(generator, EPOCHS, test_input=evaluation_noise, save_path='gan_evaluation')
generator.save('gan_model.h5')

# Additionally, you can perform fine-tuning by retraining the GAN with different settings or datasets.
# You can experiment with hyperparameter tuning, generator/discriminator architecture changes, etc.

# Example of fine-tuning by training for additional epochs:
# FINE_TUNING_EPOCHS = 50
# train_gan(generator, discriminator, gan, train_data, FINE_TUNING_EPOCHS, BATCH_SIZE, NOISE_DIM)


In [ ]:
import numpy as np
NOISE_DIM = 100
# Step 7: Generating New Car Images

# Use the generator to generate new car images.
num_generated_samples = 100
generated_noise = np.random.normal(0, 1, size=(num_generated_samples, NOISE_DIM))
generated_images = generator.predict(generated_noise)

# Post-process the generated images to denormalize pixel values to [0, 255]
generated_images = ((generated_images + 1) * 127.5).astype(np.uint8)

# Save the generated images
output_folder = 'generated_cars'
os.makedirs(output_folder, exist_ok=True)

for i in range(num_generated_samples):
    img = Image.fromarray(generated_images[i])
    img.save(f'{output_folder}/generated_car_{i:04d}.png')


NameError: name 'generator' is not defined

In [ ]:
# Step 8: Load Trained Generator Model

def load_generator_model(model_path):
    generator = make_generator_model()
    generator.load_weights(model_path)
    return generator

# Example usage to load the trained generator model
saved_model_path = 'gan_model.h5'
loaded_generator = load_generator_model(saved_model_path)

# Generating new car images using the loaded generator
num_generated_samples = 100
generated_noise = np.random.normal(0, 1, size=(num_generated_samples, NOISE_DIM))
generated_images = loaded_generator.predict(generated_noise)

# Post-process the generated images to denormalize pixel values to [0, 255]
generated_images = ((generated_images + 1) * 127.5).astype(np.uint8)

# Save the generated images
output_folder = 'generated_cars'
os.makedirs(output_folder, exist_ok=True)

for i in range(num_generated_samples):
    img = Image.fromarray(generated_images[i])
    img.save(f'{output_folder}/generated_car_{i:04d}.png')


In [ ]:
import matplotlib.pyplot as plt

# Function to visualize a sample of generated images
def visualize_generated_images(generator, noise_dim=100, num_samples=16, save_path='generated_samples.png'):
    noise = np.random.normal(0, 1, size=(num_samples, noise_dim))
    generated_images = generator.predict(noise)
    generated_images = (generated_images + 1) * 0.5  # Rescale images to [0, 1]

    fig = plt.figure(figsize=(8, 8))
    for i in range(num_samples):
        plt.subplot(4, 4, i+1)
        plt.imshow(generated_images[i])
        plt.axis('off')
    plt.savefig(save_path)
    plt.show()

# Visualize a sample of generated images
visualize_generated_images(generator, noise_dim=100, num_samples=16, save_path='generated_samples.png')


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.applications.inception_v3 import preprocess_input
from PIL import Image
import os
from scipy.linalg import sqrtm

# Function to preprocess images and get feature vectors using InceptionV3
def preprocess_images(images):
    images = preprocess_input(images)
    return images

# Function to compute FID
def compute_fid(real_images, generated_images, batch_size=64):
    real_images = preprocess_images(real_images)
    generated_images = preprocess_images(generated_images)

    # Load InceptionV3 model for feature extraction
    module = hub.load("https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4")
    
    # Get features for real and generated images
    real_features = module(real_images)
    generated_features = module(generated_images)

    # Ensure the same number of features for both sets of images
    min_num_features = min(real_features.shape[0], generated_features.shape[0])
    real_features = real_features[:min_num_features]
    generated_features = generated_features[:min_num_features]

    # Compute the covariance matrix
    covariance_real = np.cov(real_features, rowvar=False)
    covariance_generated = np.cov(generated_features, rowvar=False)

    # Compute the squared Frobenius norm between covariances
    diff = real_features - generated_features
    fid_score = np.trace(covariance_real + covariance_generated - 2 * sqrtm(np.dot(covariance_real, covariance_generated)))
    return fid_score

# Function to load and preprocess real images from a directory
def load_real_images(directory, target_size=(128, 128)):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(directory, filename)
            image = Image.open(image_path)
            image = image.resize(target_size)  # Resize the image to a target size
            image = np.array(image)  # Convert image to NumPy array
            image = image.astype(np.float32)  # Convert pixel values to float32
            image = (image / 255.0) * 2 - 1  # Normalize pixel values to [-1, 1]
            images.append(image)
    return np.array(images)

# Generating new car images for evaluation
num_evaluation_samples = 1000
evaluation_noise = np.random.normal(0, 1, size=(num_evaluation_samples, 100))
generated_images = generator.predict(evaluation_noise)

# Load and preprocess real images
real_images_directory = "cars"
real_images = load_real_images(real_images_directory)

# Compute FID
fid = compute_fid(real_images, generated_images)
print("Frechet Inception Distance (FID):", fid)


# Report

**Detailed Report Outlining Steps and Design Choices for GAN:**

**1. Introduction:**

The objective of this project is to design and implement a Generative Adversarial Network (GAN) to generate new images of cars. GANs are powerful deep learning models used for generative tasks, particularly in image generation. The GAN architecture consists of two neural networks, the generator, and the discriminator, which are trained in an adversarial manner to produce realistic images.

**2. Dataset:**

For this project, we used a dataset containing images of cars. The dataset consists of a large collection of car images with varying viewpoints, colors, and styles. A diverse dataset is crucial to ensure the generator learns to produce a wide variety of realistic car images.

**3. GAN Architecture:**

The GAN architecture consists of two main components: the generator and the discriminator.

- **Generator:** The generator takes random noise as input and transforms it into a generated image. It consists of a series of layers, such as dense layers and convolutional transpose layers, that gradually upsample the noise to produce a 128x128x3 image. The final activation function of the generator is typically 'tanh', which scales the pixel values between -1 and 1 to match the range of real images.

- **Discriminator:** The discriminator is responsible for distinguishing between real images and generated images. It takes a 128x128x3 image as input and predicts whether the image is real or fake. The discriminator consists of convolutional layers, followed by dense layers, and the final activation function is typically 'sigmoid', which produces a probability score.

**4. Hyperparameters:**

Choosing appropriate hyperparameters is essential for the successful training of the GAN. Here are the hyperparameters used in this project:

- `noise_dim`: The size of the noise vector, which is the input to the generator. In this project, we used `noise_dim = 100`.

- `learning_rate`: The learning rate used during optimization. A typical value for the Adam optimizer is `learning_rate = 0.0002`.

- `batch_size`: The number of samples in each batch used during training. We used `batch_size = 64`.

- `epochs`: The number of training epochs, which determines how many times the entire dataset is passed through the GAN during training. We used `epochs = 100`.

**5. Training Process:**

The training process of the GAN involves iteratively updating the discriminator and the generator. Here are the main steps:

- **Step 1: Define the GAN architecture:** Define the generator and the discriminator models using TensorFlow/Keras.

- **Step 2: Compile the GAN model:** Compile the GAN model using the Adam optimizer and binary cross-entropy loss.

- **Step 3: Load the real images:** Load and preprocess the real images from the car dataset. Ensure the images are resized to 128x128 pixels and normalized between -1 and 1.

- **Step 4: Generate training data for the generator:** Generate random noise vectors as the training data for the generator. Each noise vector is of size `noise_dim`.

- **Step 5: Training loop:** For a fixed number of epochs, iterate over the training data and update the discriminator and generator in an adversarial manner.

- **Step 6: Adversarial Training:** Train the discriminator using real images and the generated images. The discriminator is updated with both positive (real) and negative (generated) samples. Use binary cross-entropy loss to update the discriminator.

- **Step 7: Train the Generator:** Train the generator to generate more realistic images that can deceive the discriminator. Update the generator using the discriminator's feedback to minimize the binary cross-entropy loss.

- **Step 8: Evaluate the generator:** After training, evaluate the generator by generating new images using random noise vectors. Save and visualize the generated images to assess the quality of the model.

**6. Rationale Behind Design Choices:**

- **Architecture Choice:** We chose a simple yet effective architecture for the generator and discriminator. The generator architecture gradually upsamples the noise to produce realistic 128x128 images, while the discriminator has a structure to effectively classify real and generated images.

- **Activation Functions:** We used 'tanh' as the activation function for the generator's final layer to ensure pixel values are within the range [-1, 1]. For the discriminator, 'sigmoid' activation provides probability scores for distinguishing real and fake images.

- **Hyperparameters:** We set the hyperparameters based on common practices and experimentation. The learning rate, batch size, and number of epochs were tuned to achieve a balance between training time and the quality of generated images.

- **Training Process:** The adversarial training process is crucial for GANs. By iteratively updating the generator and discriminator, the GAN converges to a point where the generator produces realistic images, and the discriminator is unable to distinguish between real and generated images.

**7. Conclusion:**

In conclusion, we successfully designed and implemented a GAN to generate new images of cars. The GAN was trained using a dataset of real car images and random noise vectors as input to the generator. The GAN learned to generate realistic images that resembled the distribution of the real dataset. The choice of architecture and hyperparameters played a crucial role in the GAN's performance. The generated images can be used for various applications, such as data augmentation or creating synthetic datasets for training other machine learning models.

# Deploying GAN to cloud based service

Deploying a GAN to a cloud-based service involves setting up a server or cloud instance that can host the GAN model and serve requests for generating new images. Here's a brief explanation of the steps to deploy a GAN to a cloud-based service:

**1. Choose a Cloud Service Provider:**
Select a cloud service provider that suits your requirements, such as Amazon Web Services (AWS), Google Cloud Platform (GCP), Microsoft Azure, or others. Consider factors like cost, available resources, and ease of deployment when making your choice.

**2. Set Up a Virtual Machine (VM) or Container:**
Create a virtual machine (VM) or container to host your GAN model. VMs provide full flexibility, while containers offer a more lightweight and portable solution.

**3. Install Dependencies:**
Install all the necessary dependencies and libraries required to run the GAN model on the cloud instance. This includes TensorFlow/Keras, any specific libraries used in the GAN code, and the required GPU drivers (if using GPUs for accelerated computation).

**4. Upload GAN Model and Weights:**
Upload the trained GAN model and its weights to the cloud instance. This can be done by copying the model files from your local machine to the cloud instance.

**5. Expose an API Endpoint:**
Set up an API endpoint on the cloud instance that can receive requests for generating new images. This API endpoint will accept input (e.g., noise vectors) and return the generated images.

**6. Create API Server:**
Build a simple web server (using Flask, Django, FastAPI, etc.) that listens for incoming requests on the API endpoint. This server will load the GAN model and use it to generate new images based on the input received.

**7. Implement Generation Logic:**
In the web server, implement the logic to process the input (e.g., noise vectors) and generate images using the loaded GAN model. Make sure to preprocess the input and post-process the output images as needed.

**8. Handle Multiple Requests:**
Ensure that the web server can handle multiple requests simultaneously. Consider using a queue system or asynchronous processing to handle multiple requests efficiently.

**9. Deploy the Server:**
Deploy the web server on the cloud instance and make it accessible via a public IP or domain. This is usually done using the cloud service provider's interface or command-line tools.

**10. Test and Monitor:**
Test the deployed GAN service by sending requests and verifying that it generates images as expected. Monitor the server's performance, resource usage, and any errors or exceptions to ensure smooth operation.

**11. Scale as Needed:**
Depending on the demand, scale up the cloud instance or use load balancers to distribute requests across multiple instances for improved performance and reliability.

**12. API Documentation:**
Optionally, create documentation for your GAN API, specifying the input format, expected responses, and any usage guidelines.

By following these steps, you can deploy your GAN to a cloud-based service, allowing others to access and use your model for generating new images via simple API requests.